# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

/Users/thomasnguyen/opt/anaconda3/envs/project3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """

    columns = [
        'Title',
        'Year',
        'Synopsis',
        'Review'
    ]

    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_eng.columns = columns
    df_eng['Original Language'] = ['English' for _ in range(len(df_eng['Review']))]

    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr.columns = columns
    df_fr['Original Language'] = ['French' for _ in range(len(df_fr['Review']))]

    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp.columns = columns
    df_sp['Original Language'] = ['Spanish' for _ in range(len(df_sp['Review']))]

    
    concat_pd = pd.concat([df_eng, df_fr, df_sp])
    
    return concat_pd

df = preprocess_data()

In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
4,Amores perros,2000,Tres historias se entrelazan en esta película ...,"""Amores perros es una película intensa y conmo...",Spanish
5,Águila Roja,(2009-2016),Esta serie de televisión española sigue las av...,"""Águila Roja es una serie aburrida y poco inte...",Spanish
7,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido...",Spanish
8,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",French
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
1,Intouchables,2011,Ce film raconte l'histoire de l'amitié improba...,"""Intouchables est un film incroyablement touch...",French
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English
5,Le Dîner de Cons,1998,Le film suit l'histoire d'un groupe d'amis ric...,"""Je n'ai pas aimé ce film du tout. Le concept ...",French
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# load translation models and tokenizers
from transformers import MarianMTModel, MarianTokenizer

fr_en_model_name = 'Helsinki-NLP/opus-mt-fr-en'
es_en_model_name = 'Helsinki-NLP/opus-mt-es-en'
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)


/Users/thomasnguyen/opt/anaconda3/envs/project3/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors='pt')

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

In [6]:
txt = 'El Incidente es una película aburrida y sin sustos que no logra crear una atmósfera de miedo o tensión. La actuación es mediocre y la trama es predecible y poco emocionante.'

print(translate(txt, es_en_model, es_en_tokenizer))

The Incident is a boring and frightless film that fails to create an atmosphere of fear or tension. The performance is mediocre and the plot is predictable and unexciting.


In [7]:
txt = "Ce film est un gâchis complet. Les personnages sont sans intérêt, l'intrigue est confuse et les effets spéciaux sont mauvais. Le film essaie d'être intelligent et cool, mais échoue lamentablement. Évitez ce film à tout prix."

print(translate(txt, fr_en_model, fr_en_tokenizer))

This film is a complete mess. The characters are of no interest, the plot is confused and the special effects are bad. The film tries to be smart and cool, but fails lamentably. Avoid this film at all costs.


In [8]:
# filter reviews in French and translate to English
fr_reviews = df.loc[df['Original Language'] == 'French', 'Review'].tolist()
df.loc[df['Original Language'] == 'French', 'Review'] = [translate(txt, fr_en_model, fr_en_tokenizer) for txt in fr_reviews]

# filter synopsis in French and translate to English
fr_synopsis = df.loc[df['Original Language'] == 'French', 'Synopsis'].tolist()
df.loc[df['Original Language'] == 'French', 'Synopsis'] = [translate(txt, fr_en_model, fr_en_tokenizer) for txt in fr_synopsis]

# filter reviews in Spanish and translate to English
es_reviews = df.loc[df['Original Language'] == 'Spanish', 'Review'].tolist()
df.loc[df['Original Language'] == 'Spanish', 'Review'] = [translate(txt, es_en_model, es_en_tokenizer) for txt in es_reviews]

# filter synopsis in Spanish and translate to English
es_synopsis = df.loc[df['Original Language'] == 'Spanish', 'Synopsis'].tolist()
df.loc[df['Original Language'] == 'Spanish', 'Synopsis'] = [translate(txt, es_en_model, es_en_tokenizer) for txt in es_synopsis]


In [9]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
3,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",Spanish
0,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",French
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
7,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",Spanish
2,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",French
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",English
7,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete disappointment. The j...",French
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",English
9,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",Spanish


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [10]:
# load sentiment analysis model

model_name = 'finiteautomata/bertweet-base-sentiment-analysis'
sentiment_classifier = pipeline(model=model_name)

def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    decoder = {
        'POS': 'Positive',
        'NEG': 'Negative',
    }

    list_sentiment = classifier([str(text)])


    return decoder[list_sentiment[0]['label']]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [11]:
analyze_sentiment("I'm down bad", sentiment_classifier)

'Negative'

In [15]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
# df.assign(Sentiment=lambda x: analyze_sentiment(df['Review'], sentiment_classifier))
df = df.assign(Sentiment=df['Review'].apply(lambda x: analyze_sentiment(x, sentiment_classifier)))



In [16]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",English,Positive
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English,Negative
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English,Negative
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English,Positive
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English,Negative
8,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",Spanish,Negative
4,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",Spanish,Positive
6,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",French,Negative
2,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",French,Positive
0,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",French,Positive


In [17]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)